# Test viscous liquid transfer

The objective of this Jupyter is to guide the user through the process of obtaining the liquid handling parameters for OT2 pippetes that accutately transfer volumes of viscous liquids. The process is based on a gravimetric method where the volume transfered with the OT2 pipettes is compared with the mass transfered during pipetting, allowing to compute a relative transfer error for each set of liquid handling parameters. The liquid handling parameters are tuned by a multi-objective Bayesian Optimization algorithm to minimize the relative transfer error and time of transfer.


## Initialization of python API, transfer function and OT2 Deck Setup

In [ ]:
#Import relevant python packages
import os
import json
import opentrons.execute
import pandas as pd
import numpy as np
import time


In [ ]:
def transfer_viscous_liquid(liquid, volume, source, destination, pipette, protocol, distance, new_tip=True):
    """Function to transfer viscous liquids using a OT2 robot (v2).
    Args:
        liquid: key in liquids_dict for liquid handling parameter definitions.
        volume: target volume to be transfered.
        source/destination: point object of OT2 that define place of aspiration and dispense respectiveley.
        pipette: OT2 InstrumentContext object.
        protocol: ProtocolContext object. 
        distance: height of liquid in source vial in mm
        new_tip specifies: True if new tip is required for the transfer.
    """
    if new_tip == True: pipette.pick_up_tip()
    if 'P1' in str(pipette):
        pipette_name = 'p1000'
    elif 'P3'in str(pipette):
        pipette_name = 'p300'
    if pipette.has_tip == False:
        raise Exception("Can't aspirate liquid with no tip")
    pipette.aspirate(volume, source.bottom(distance), rate = liquids_dict[liquid][pipette_name]['aspiration_rate']/pipette.flow_rate.aspirate)
    protocol.delay(seconds =  liquids_dict[liquid][pipette_name]['delay_aspirate'])
    if liquids_dict[liquid][pipette_name]['touch_tip_aspirate'] == True:
        pipette.touch_tip()
    pipette.dispense(volume, destination.top(-5), rate = liquids_dict[liquid][pipette_name]['dispense_rate']/pipette.flow_rate.dispense)
    protocol.delay(seconds = liquids_dict[liquid][pipette_name]['delay_dispense'])
    if liquids_dict[liquid][pipette_name]['blow_out_rate'] > 0:
        pipette.flow_rate.blow_out = liquids_dict[liquid][pipette_name]['blow_out_rate']
        pipette.blow_out()
        pipette.flow_rate.blow_out = pipette.flow_rate.aspirate 
        protocol.delay(seconds = liquids_dict[liquid][pipette_name]['delay_blow_out'])
    if liquids_dict[liquid][pipette_name]['touch_tip_dispense'] == True:
        pipette.touch_tip()   
    if new_tip == True:
        pipette.drop_tip()


def ccf_parameters(liquid,liquids_dict, bounds=[0.1,1.25]):
    """
    Function to create dictionary containing the values for aspiration and dispense rates
    used during the exploration stage 
    Args:
        liquid (str): Name of liquid to be tested 
        liquids_dict (dict): Dictionary cotnaining the liquid handling parameters of tested liquid
        bounds (list): List composed of the upper and lower factors to calculate the mihimum and 
        maximum values for the prametric space
    Returns:
        parameters_dict (dict): Dictionary containing the liquid handing parameters to be tested during
        the exploration step
    """
 
    aspiration_rates = [liquids_dict[liquid]['p1000']["aspiration_rate"]*bounds[1],liquids_dict[liquid]['p1000']["aspiration_rate"]*bounds[1],liquids_dict[liquid]['p1000']["aspiration_rate"]*bounds[0],liquids_dict[liquid]['p1000']["aspiration_rate"]*bounds[0]]
    dispense_rates =  [liquids_dict[liquid]['p1000']["dispense_rate"]*bounds[1],liquids_dict[liquid]['p1000']["dispense_rate"]*bounds[0],liquids_dict[liquid]['p1000']["dispense_rate"]*bounds[1],liquids_dict[liquid]['p1000']["dispense_rate"]*bounds[0]] 
    parameters_dict = {
        "aspiration_rate": aspiration_rates, 
        "dispense_rate": dispense_rates,
    }
     
   
    return parameters_dict


def update_dict(liquid, new_parameters_dict,repeat, liquids_dict):
    """Function that updates the liquid handling parameters contained in a dictionary with each transfer 
    performed during the exploration step
    Args: 
        liquid (str): Name of liquid to be tested 
        new_parameters (dict): Dictionary generated with ccf_parameters
        repeat (int: Integer ranging from 1-4 that poitns to the iteration number in the exploration step
        to be tested
        liquids_dict (dict): Dictionary cotnaining the liquid handling parameters of tested liquid
    """

    liquids_dict[liquid]['p1000']["aspiration_rate"] = new_parameters_dict["aspiration_rate"][repeat-1]
    liquids_dict[liquid]['p1000']["dispense_rate"] = new_parameters_dict["dispense_rate"][repeat-1]   
    liquids_dict[liquid]['p1000']["delay_aspirate"] = new_parameters_dict["delay_aspirate"][repeat-1]
    liquids_dict[liquid]['p1000']["delay_dispense"] = new_parameters_dict["delay_dispense"][repeat-1]    

In [ ]:
#Initialization of API and deck setup
protocol = opentrons.execute.get_protocol_api('2.11')
protocol.home()
tiprack_1000 = protocol.load_labware('opentrons_96_tiprack_1000ul', 11)
tiprack_300=  protocol.load_labware('opentrons_96_tiprack_300ul', 8)
pipettes = {'p1000' : protocol.load_instrument('p1000_single_gen2', 'left', tip_racks=[tiprack_1000]), 'p300' : protocol.load_instrument('p300_multi_gen2', 'right', tip_racks=[tiprack_300])}
source = protocol.load_labware('amdm_12_wellplate_30000ul',6) 
# destination = protocol.load_labware('amdm_12_wellplate_30000ul',6)

In [ ]:
#Stablish starting pippette tips locations
pipettes['p1000'].starting_tip = tiprack_1000.well('H6')
pipettes['p300'].starting_tip = tiprack_300.well('F10')


In [ ]:
#Stablish initial height of liquid on the source vial
liquid_level = 51

## Viscous liquid protocol: Coarse approximation of pipetting parameters

The first step is to obtain approximate values of aspiration and dispense rates that can be used to initialize the ;iqiid transfer such as aspiration and dispense rates. 

In [ ]:
liquid = 'Viscosity_std_1275'
density = 0.8736
pipette = 'p1000'
volume = 1000


In [ ]:
#This commands will aspirate 1000ul liquid at standard flow_rate.aspirate of pippette. A timer well be started just before aspiration starts
pipettes[pipette].pick_up_tip()
pipettes[pipette].move_to(source.wells_by_name()['A1'].bottom(liquid_level-15))
start = time.time()
pipettes[pipette].aspirate(volume,rate = 1)

In [ ]:
#Run this cell when no further flow of liquid into the pipette tip is observed. Calculates an approximate flow rate for 
#aspiration
finish = time.time()
t_aspirate = finish-start
flow_rate_aspirate = volume/t_aspirate
flow_rate_aspirate

In [ ]:
#Dispense volume 
pipettes[pipette].dispense(volume,rate = (flow_rate_aspirate/2)/pipettes[pipette].flow_rate.aspirate)

In [ ]:
#This command will clear out remaining liquid in the tip if the dispense was incomplete.
pipettes[pipette].move_to(source.wells_by_name()['A1'].top())
protocol.delay(5)
pipettes[pipette].home_plunger()
protocol.delay(seconds=10)
pipettes[pipette].blow_out(location = source.wells_by_name()['A1'].top())
pipettes[pipette].touch_tip(location = source.wells_by_name()['A1'])

pipettes[pipette].home_plunger()
protocol.delay(seconds=10)
pipettes[pipette].blow_out(location = source.wells_by_name()['A1'].top())
pipettes[pipette].touch_tip(location = source.wells_by_name()['A1'])

pipettes[pipette].home_plunger()
protocol.delay(seconds=10)
pipettes[pipette].blow_out(location = source.wells_by_name()['A1'].top())
pipettes[pipette].touch_tip(location = source.wells_by_name()['A1'])
pipettes[pipette].move_to(source.wells_by_name()['A1'].top())


In [ ]:
#New dataframe
df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate','blow_out_rate', 'delay_aspirate',  'delay_dispense', 'delay_blow_out','touch_tip_aspirate', 'touch_tip_dispense', 'density', 'time','mi', 'mf', 'm', '%error', 'Transfer_Observation', 'Comment'])
df = df.astype({'liquid':str,'pipette':str,"touch_tip_aspirate":bool,"touch_tip_dispense":bool,'Transfer_Observation':str,'Comment':str})


In [ ]:
liquids_dict = {
  liquid :{
        "p1000": {
            "aspiration_rate": flow_rate_aspirate, 
            "dispense_rate": flow_rate_aspirate,
            "blow_out_rate" : 0, 
            "delay_aspirate" : 5, 
            "delay_dispense" : 5, 
            "delay_blow_out" : 0,
            "touch_tip_aspirate": True, 
            "touch_tip_dispense" : False,
            },

        "p300": {
            "aspiration_rate": 25 , 
            "dispense_rate": 12.5, 
            "blow_out_rate" : 0 , 
            "delay_aspirate" : 3, 
            "delay_dispense" : 3, 
            "delay_blow_out" : 0,
            "touch_tip_aspirate": True, 
            "touch_tip_dispense" : False,
        }
    }

}
print(liquids_dict[liquid][pipette])

Gravimetric test with approximated flow rate obtained during initialization

In [ ]:
pipette = 'p1000'
volume = 1000
mi = 21.3625
if pipettes[pipette].has_tip == False:
    pipettes[pipette].pick_up_tip()
start = time.time()
transfer_viscous_liquid(liquid, volume, source.wells_by_name()['A1'], source.wells_by_name()['A4'], pipettes[pipette], protocol, liquid_level-15, new_tip=False)
pipettes[pipette].move_to(source.wells_by_name()['A1'].top())
df = df.append(liquids_dict[liquid][pipette], ignore_index = True)
finish = time.time()
time_m = finish - start

In [ ]:
mf = 29.6300
m = mf-mi
error = (m-density*volume/1000)/(density/1000*volume)*100
df.iloc[-1,-7] = time_m
df.iloc[-1,2] = volume
df.iloc[-1, 0] = liquid
df.iloc[-1, 1] = pipette
df.iloc[-1,-8] = density
df.iloc[-1,-6] = mi
df.iloc[-1,-5] = mf
df.iloc[-1, -4] = m
df.iloc[-1,-3]= error


In [ ]:
#Update liquid level
liquid_level = liquid_level - 2*volume/1000

In [ ]:
#Observe error made
df

In [ ]:
if -5<df.iloc[-1,-3]<5:
    value = 'Within tolerance'
elif df.iloc[-1,-3]>5:
    value = 'Excess aspiration and dispense'
else:
    value = 'Incomplete aspiration and dispense'

In [ ]:
#Assign category of observation of transfer such as Incomplete Dispense, Incomplete Aspiration, 
#Incomplete Aspiration and Dispense, Complete Transfer. 
#Comment if any unexpected exprimental mistakes or changes were performed that have to be taken into account.
df.iloc[-1,-2]= value
df.iloc[-1,-1]= ''

In [ ]:
pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
pipettes['p1000'].move_to(source.wells_by_name()['A1'].top())


Continue with gravimetric transfers of exploration step, run the following code until repeat = 4

In [ ]:
repeat = 1
new_parameters = ccf_parameters(liquid, liquids_dict=liquids_dict)
print(new_parameters)

In [ ]:
update_dict(liquid, new_parameters,repeat, liquids_dict)
repeat+=1
print(liquids_dict[liquid][pipette])

In [ ]:
pipette = 'p1000'
volume  = 300
mi = mf
if pipettes[pipette].has_tip == False:
    pipettes[pipette].pick_up_tip()
start = time.time()
transfer_viscous_liquid(liquid, volume, source.wells_by_name()['A1'], source.wells_by_name()['A4'], pipettes[pipette], protocol, liquid_level-15, new_tip=False)
pipettes[pipette].move_to(source.wells_by_name()['A1'].top())
df = df.append(liquids_dict[liquid][pipette], ignore_index = True)
finish = time.time()
time_m = finish - start

In [ ]:
mf = 30.
m = mf-mi
error = (m-density*volume/1000)/(density/1000*volume)*100
df.iloc[-1,-7] = time_m
df.iloc[-1,2] = volume
df.iloc[-1, 0] = liquid
df.iloc[-1, 1] = pipette
df.iloc[-1,-8] = density
df.iloc[-1,-6] = mi
df.iloc[-1,-5] = mf
df.iloc[-1, -4] = m
df.iloc[-1,-3]= error


In [ ]:
#Update liquid level
liquid_level = liquid_level - 2*volume/1000
liquid_level

In [ ]:
#Observe error made
df

In [ ]:
#Assign category of observation of transfer such as Incomplete Dispense, Incomplete Aspiration, 
#Incomplete Aspiration and Dispense, Complete Transfer. 
#Comment if any unexpected exprimental mistakes or changes were performed that have to be taken into account.
df.iloc[-1,-2]= 'Excess aspiration and dispense'
df.iloc[-1,-1]= ''

In [ ]:
pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])


pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
pipettes['p1000'].move_to(source.wells_by_name()['A1'].top())


# ML guided optimization for gravimetric calibration of viscous liquid transfer

The following cells contain the code required to implemenet the gravimetric analysis of volume transfer of a specific viscousl iquid. User only needs to update dictionary values for liquid handling parameters, input the target volume, density of the liquid and mass of vials before and after a dispense.

In [ ]:
liquid = 'Viscosity_std_1275'
density = 0.8736
pipette = 'p1000'
volume = 1000
liquid_level = 51


If the experiment is the continutation of a previous initialization load the dataframe to record transfer parameters.

In [ ]:
#Load dataframe
df=pd.read_csv('')


Update liquid handling parameters with suggestions from BO model

In [ ]:
liquids_dict = {
  liquid :{
        "p1000": {
            "aspiration_rate": 26.604085, 
            "dispense_rate": 4.049484,
            "blow_out_rate" : 0, 
            "delay_aspirate" : 5, 
            "delay_dispense" : 5, 
            "delay_blow_out" : 0,
            "touch_tip_aspirate": True, 
            "touch_tip_dispense" : False,
            },

        "p300": {
            "aspiration_rate": 25 , 
            "dispense_rate": 12.5, 
            "blow_out_rate" : 0 , 
            "delay_aspirate" : 3, 
            "delay_dispense" : 3, 
            "delay_blow_out" : 0,
            "touch_tip_aspirate": True, 
            "touch_tip_dispense" : False,
        }
    }

}
print(liquids_dict[liquid][pipette])

Transfer viscous liquds, input pippette name (pipette), desired volume (volume) to be dispensed in ul, liquid dictonary key string (liquid), density (density) and initial vial mass (mi). The code will register the liquid handling parameters used into the dataframe  

In [ ]:
volume = 300
mi = mf
if pipettes[pipette].has_tip == False:
    pipettes[pipette].pick_up_tip()
start = time.time()
transfer_viscous_liquid(liquid, volume, source.wells_by_name()['A1'], source.wells_by_name()['A4'], pipettes[pipette], protocol, liquid_level-15, new_tip=False)
pipettes[pipette].move_to(source.wells_by_name()['A1'].top())
df = df.append(liquids_dict[liquid][pipette], ignore_index = True)
finish = time.time()
time_m = finish - start

 Input mass of vial after transfer (mf). Code will calculate the relative error of transfer

In [ ]:
mf = 27.5176
m = mf-mi
error = (m-density*volume/1000)/(density*volume/1000)*100
df.iloc[-1,-7] = time_m
df.iloc[-1,2] = volume
df.iloc[-1, 0] =  liquid
df.iloc[-1, 1] = pipette
df.iloc[-1,-8] = density
df.iloc[-1,-6] = mi
df.iloc[-1,-5] = mf
df.iloc[-1, -4] = m
df.iloc[-1,-3]= error


In [ ]:
#Update liquid level
liquid_level = liquid_level - 2*(m/density)
liquid_level

In [ ]:
#Observe error made
df.iloc[-5:]

In [ ]:
if -5<df.iloc[-1,-3]<5:
    value = 'Within Tolerance'
elif df.iloc[-1,-3]>5:
    value = 'Excess Aspiration and Dispense'
else:
    value = 'Incomplete Aspiration and Dispense'
    

In [ ]:
#Assign category of observation of transfer such as Incomplete Dispense, Incomplete Aspiration, 
#Incomplete Aspiration and Dispense, Excess aspiration and dispense, Within Tolerance. 
#Comment if any unexpected exprimental mistakes or changes were performed that have to be taken into account.
df.iloc[-1,-2]= value
df.iloc[-1,-1]= ''
df.iloc[-5:]

In [ ]:
for i in range(2):
    
    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())

    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
    pipettes['p1000'].home_plunger()
    protocol.delay(seconds=10)

    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
    pipettes['p1000'].home_plunger()
    protocol.delay(seconds=10)

    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
    pipettes['p1000'].home_plunger()
    protocol.delay(seconds=10)
 

Save data, can be used at any time.

In [ ]:
df.to_csv('BOTorch_optimization'+'_'+ 'exp3' +'_'+liquid+'_all_rawdata.csv', index = False)

In [ ]:
df.to_csv('liquid_4_selina_initialization.csv')


## Auxiliary code

In [ ]:
#Shut down sequence

pipettes[pipette].drop_tip()

protocol.home()

In [ ]:
#Clean residue from pipette tip
for i in range(2):

    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

    pipettes['p1000'].home_plunger()
    protocol.delay(seconds=10)
    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

    pipettes['p1000'].home_plunger()
    protocol.delay(seconds=10)
    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
    pipettes['p1000'].move_to(source.wells_by_name()['A1'].top())
